# Athena SQL Model

This example will create an athena table for [Jan 2017 taxi dataset](https://aws.amazon.com/blogs/big-data/build-a-data-lake-foundation-with-aws-glue-and-amazon-s3/).  You can improve performance if you convert into a parquet format.

Configure your notebook role with permissions to [query data from athena](https://aws.amazon.com/blogs/machine-learning/run-sql-queries-from-your-sagemaker-notebooks-using-amazon-athena/) and access the s3 staging bucket.

## Install libraries

Install the [Athena library](https://pypi.org/project/PyAthena/) for python and [tqdm](https://tqdm.github.io/)

In [ ]:
import sys
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install -U pandas
!{sys.executable} -m pip install -U PyAthena[Pandas]==1.11.2
!{sys.executable} -m pip install -U tqdm
!{sys.executable} -m pip install -U sagemaker

### Restart Kernel

Now that you have upgraded SageMaker you need to restart the kernel by clicking menu: `Kernel -> Restart & Clear Output`.

Once restarted, run the next cell to check you have version starting with `2.x`

In [ ]:
import sys
!{sys.executable} -m pip show sagemaker

## Import Data

Create an anthena database and external table for the imported nyc bit dataset.

In [ ]:
import boto3
import sagemaker

# Initialize the boto session in us-east-1 region
boto_session = boto3.session.Session(region_name='us-east-1')
region = boto_session.region_name
bucket = sagemaker.session.Session(boto_session).default_bucket()

# Get the athena staging dir andtable
s3_staging_dir = 's3://{}/athena'.format(bucket)
db_name = 'nyc_taxi'
table_name = '{}.taxi_csv'.format(db_name)

print('s3 staging dir: {}'.format(s3_staging_dir))
print('athena table: {}'.format(table_name))

Make the bucket if it doesn't exist

In [ ]:
!aws s3 mb s3://$bucket --region $region

Query the nyc taxi dataset using [PandasCursor](https://pypi.org/project/PyAthena/#pandascursor) for improved performance

In [ ]:
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
import pandas as pd

cursor = connect(s3_staging_dir=s3_staging_dir,
                 region_name=region,
                 cursor_class=PandasCursor).cursor()

In [ ]:
sql_ddl_create_table = 'CREATE DATABASE IF NOT EXISTS {};'.format(db_name)

cursor.execute(sql_ddl_create_table)
print('Status: {}, Run time: {:.2f}s'.format(cursor.state, 
    cursor.execution_time_in_millis/1000.0))

In [ ]:
sql_create_table = '''
CREATE EXTERNAL TABLE IF NOT EXISTS `{}` (
    `vendorid` bigint, 
    `lpep_pickup_datetime` string, 
    `lpep_dropoff_datetime` string, 
    `store_and_fwd_flag` string, 
    `ratecodeid` bigint, 
    `pulocationid` bigint, 
    `dolocationid` bigint, 
    `passenger_count` bigint, 
    `trip_distance` double, 
    `fare_amount` double, 
    `extra` double, 
    `mta_tax` double, 
    `tip_amount` double, 
    `tolls_amount` double, 
    `ehail_fee` string, 
    `improvement_surcharge` double, 
    `total_amount` double, 
    `payment_type` bigint, 
    `trip_type` bigint)
ROW FORMAT DELIMITED 
    FIELDS TERMINATED BY ',' 
STORED AS INPUTFORMAT 
    'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
    'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
    's3://aws-bigdata-blog/artifacts/glue-data-lake/data/'
TBLPROPERTIES (
    'columnsOrdered'='true', 
    'compressionType'='none', 
    'skip.header.line.count'='1')
'''.format(table_name)

cursor.execute(sql_create_table)
print('Status: {}, Run time: {:.2f}s'.format(cursor.state, 
    cursor.execution_time_in_millis/1000.0))

In [ ]:
data_sql = '''
SELECT 
    total_amount, fare_amount, lpep_pickup_datetime, lpep_dropoff_datetime, trip_distance 
FROM {} WHERE total_amount is not null;
'''.format(table_name)
print('Querying...', data_sql)

data_df = cursor.execute(data_sql).as_pandas()
print('Status: {}, Run time: {:.2f}s, Data scanned: {:.2f}MB, Records: {:,}'.format(cursor.state, 
    cursor.execution_time_in_millis/1000.0, cursor.data_scanned_in_bytes/1024.0/1024.0, data_df.shape[0]))

data_df.head()

Performance some simple feature engineering

In [ ]:
# Add some date features
data_df['lpep_pickup_datetime'] = data_df['lpep_pickup_datetime'].astype('datetime64[ns]')
data_df['lpep_dropoff_datetime'] = data_df['lpep_dropoff_datetime'].astype('datetime64[ns]')
data_df['duration_minutes'] = (data_df['lpep_dropoff_datetime'] - data_df['lpep_pickup_datetime']).dt.seconds/60
data_df['hour_of_day'] = data_df['lpep_pickup_datetime'].dt.hour
data_df['day_of_week'] = data_df['lpep_pickup_datetime'].dt.dayofweek
data_df['week_of_year'] = data_df['lpep_pickup_datetime'].dt.weekofyear
data_df['month_of_year'] = data_df['lpep_pickup_datetime'].dt.month

In [ ]:
# Exclude any outliers
data_df = data_df[(data_df.total_amount > 0) & (data_df.total_amount < 200) & 
                  (data_df.duration_minutes > 0) & (data_df.duration_minutes < 120) & 
                  (data_df.trip_distance > 0) & (data_df.trip_distance < 1000)].dropna()
print(data_df.shape)
data_df.head()

## Train Model

Build an XGBoost model to predict the total amount based on some fields

In [ ]:
import boto3 
import sagemaker

sagemaker_session = sagemaker.session.Session(boto_session)
role = sagemaker.get_execution_role()
prefix = 'nyc-taxi'

print('bucket: {}, prefix: {}'.format(bucket, prefix))

In [ ]:
# Trip test split
from sklearn.model_selection import train_test_split

train_cols = ['total_amount', 'duration_minutes', 'trip_distance', 'hour_of_day']
train_df, val_df = train_test_split(data_df[train_cols], test_size=0.20, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.50, random_state=42)

print('split train: {}, val: {}, test: {} '.format(train_df.shape[0], val_df.shape[0], test_df.shape[0]))

In [ ]:
# Reset index and save files with target as first column
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

### Upload Data

Save train and validation as CSV with `total_amount` as first col but no headers

In [ ]:
# Drop the tpep_pickup_datetime and save
train_df.to_csv('train.csv', index=False, header=False)
val_df.to_csv('validation.csv', index=False, header=False)

In [ ]:
%%time

# Uplaod the files to s3 
s3_train_uri = sagemaker_session.upload_data('train.csv', bucket, prefix + '/data/training')
s3_val_uri = sagemaker_session.upload_data('validation.csv', bucket, prefix + '/data/validation')

Validate that we have uploaded these files succesfully

In [ ]:
!aws s3 ls $s3_train_uri 
!aws s3 ls $s3_val_uri

### Get estimator

In [ ]:
container = sagemaker.image_uris.retrieve(region=region, framework="xgboost", version="latest")
print('container: {}'.format(container))

In [ ]:
output_path = 's3://{}/{}/output'.format(bucket, prefix)
print('output: {}'.format(output_path))

xgb = sagemaker.estimator.Estimator(container,
                                    role,
                                    instance_count=1,
                                    instance_type='ml.m4.xlarge',
                                    output_path=output_path,
                                    sagemaker_session=sagemaker_session)

In [ ]:
xgb.set_hyperparameters(max_depth=9,
                        eta=0.2, 
                        gamma=4,
                        min_child_weight=300,
                        subsample=0.8,
                        silent=0,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=10000)

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=s3_train_uri, content_type="csv")
s3_input_val = sagemaker.inputs.TrainingInput(s3_data=s3_val_uri, content_type="csv")

xgb.fit({'train': s3_input_train,  'validation': s3_input_val})

### Deploy model

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name='xgb-athena-integration-endpoint')

### Evalulate Model

Get predicitons for the validation set

In [ ]:
from sagemaker.serializers import CSVSerializer
xgb_predictor.serializer = CSVSerializer()

In [ ]:
%%time

import numpy as np
from tqdm import tqdm

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in tqdm(split_array):
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
    return np.fromstring(predictions[1:], sep=',')

# Get predictions and store in df
predictions = predict(val_df[train_cols[1:]].values)
predictions = pd.DataFrame({'total_amount_predictions': predictions })

In [ ]:
# Get the abs error between predictions
pred_df = val_df.join(predictions)
pred_df['error'] = abs(pred_df['total_amount']-pred_df['total_amount_predictions'])
pred_df.sort_values('error', ascending=True).head(10)

## Create Athena UDF 

Create a [User Defined Function](https://aws.amazon.com/blogs/big-data/prepare-data-for-model-training-and-invoke-machine-learning-models-with-amazon-athena/) for the deployed endpoint so you can query directly in Athena.

In [ ]:
endpoint_name = xgb_predictor.endpoint_name
print('endpoint: {}'.format(endpoint_name))

`NOTE`: Athena ML is [in preview](https://aws.amazon.com/athena/faqs/#Preview_features).   To enable this Preview feature you need to create an Athena workgroup named `AmazonAthenaPreviewFunctionality` and run any queries attempting to federate to this connector, use a UDF, or SageMaker inference from that workgroup.

In [ ]:
workgroup_name = 'AmazonAthenaPreviewFunctionality'

!aws athena create-work-group --name $workgroup_name --region $region

Using presto [datetime](https://prestodb.io/docs/0.172/functions/datetime.html) functions with inline query, rank by absolute error.

In [ ]:
query_sql  = '''
USING FUNCTION predict_total(
  duration_minutes DOUBLE, 
  trip_distance DOUBLE, 
  hour_of_day DOUBLE) returns DOUBLE type SAGEMAKER_INVOKE_ENDPOINT
WITH (sagemaker_endpoint='{}')

SELECT 
    *, ABS(predicted_total_amount-total_amount) as error
FROM ( 
    SELECT
        *,
        predict_total(duration_minutes, trip_distance, hour_of_day) as predicted_total_amount
    FROM 
    (
        SELECT 
            total_amount,
            CAST(date_diff('minute', 
                CAST(lpep_pickup_datetime as timestamp), 
                CAST(lpep_dropoff_datetime as timestamp)) as DOUBLE) as duration_minutes,
            CAST(trip_distance as DOUBLE) as trip_distance,
            CAST(hour(CAST(lpep_pickup_datetime as timestamp)) as double) as hour_of_day
        FROM {}
        WHERE DAY(CAST(lpep_pickup_datetime as timestamp)) = {} -- Filter by day
    )
)
ORDER BY error DESC
LIMIT {};
'''.format(endpoint_name, table_name, 1, 10)
print('Querying...', query_sql)

query_df = cursor.execute(query_sql, work_group=workgroup_name).as_pandas()
print('Status: {}, Run time: {:.2f}s, Data scanned: {:.2f}MB, Records: {:,}'.format(cursor.state, 
    cursor.execution_time_in_millis/1000.0, cursor.data_scanned_in_bytes/1024.0/1024.0, query_df.shape[0]))

query_df